# Hyperparameter Optimization of Wavenet Based Model

The search uses Bayesian optimization using Gaussian Processes implemented by the gp_minimize function from scikit-optimize.

## Notebook setup

Allow editing of modules using editor (auto reloading)

In [0]:
# Needed to allow editing using PyCharm etc
%load_ext autoreload
%autoreload 2

The following cell is needed for compatibility when using both CoLab and Local Jupyter notebook. It sets the appropriate file path for the data.

In [0]:
import os
path = os.getcwd()
if path == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    BASE_PATH = '/content/gdrive/My Drive/Level-4-Project/'
    !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
elif path == 'D:\\Google Drive\\Level-4-Project\\notebooks\\wavenet':
    BASE_PATH = "D:/Google Drive/Level-4-Project/"
    
elif path == "/export/home/2192793m":
    BASE_PATH = "/export/home/2192793m/Level-4-Project/"
    
DATA_PATH = BASE_PATH + 'data/processed/wavenet/raw_reshaped/'
MODEL_PATH = BASE_PATH + 'models/wavenet/raw_reshaped/test_8/'
RESULTS_PATH = BASE_PATH + 'results/wavenet/raw_reshaped/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Obtaining file:///content/gdrive/My%20Drive/Level-4-Project
  Found existing installation: src 0.1.0
    Can't uninstall 'src'. No files were found to uninstall.
  Running setup.py develop for src


Using TensorFlow backend.


Import local packages

In [0]:
from src.data import load_data
from src.visualization import multiple_plots, visualize, plot_confusion_matrix
from src.features import make_spectrograms, process_labels, make_directory
from src.models.dilated_CNN.wavenet_kevin_mader import WaveNetClassifier
from src.data_generator.data_generator import DataGenerator

Import the remaining packages that are needed

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import classification_report, confusion_matrix
from keras import metrics
from keras import optimizers
from keras.callbacks import History, ModelCheckpoint, CSVLogger
from keras.models import load_model

In [0]:
! pip install scikit-optimize
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

## Data Loading

In [0]:
# Load in data dictionary.
# This does not load in any actual data,
# just the dictionary with the names of the files and their associated labels
with open(DATA_PATH + "index.pkl", "rb") as file:
    data = pickle.load(file)

In [0]:
Remove user C as this user is reserved for the test set
try:
    del data["C"]
except KeyError:
    print ("Key 'C' not found")

In [0]:
def convert_label_to_int(label):
    if label == "walking":
        return 0
    if label == "pushing":
        return 1
    if label == "sitting":
        return 2
    if label == "pulling":
        return 3
    if label == "circling":
        return 4
    if label == "clapping":
        return 5
    if label == "bending":
        return 6

In [0]:
labels = {}
X = []

for user_letter, actions in data.items():
    for action, results in actions.items():
        for result in results:
            for row in result:
                X.append(row)
                labels[row] = convert_label_to_int(action)

In [0]:
target_names = ["walking", "pushing", "sitting", "pulling", "circling", "clapping", "bending"]
NB_CLASSES = len(target_names)

## Hyperparameter Search

### Fixed Parameters

In [0]:
# Shape of the input data
DATA_SHAPE = (3000, 128)
# Number of folds for K-fold cross validation
NB_OF_FOLDS = 5

### Parameters to Search

In [0]:
n_filters = 64
dilation_depth = 8
activation = 'softmax'
scale_ratio = 1
kernel_size = 2
pool_size_1 = 4
pool_size_2 = 8
batch_size = 16
epochs=15

In [0]:
space = [Integer(1,8, name='nb_of_layers'),
         Real(0.001, 0.1, "log-uniform", name='learning_rate')
        ]

### K-Fold Setup


In [0]:
kf = KFold(n_splits=NB_OF_FOLDS, shuffle=True, random_state=12)

### Objective Function to Minimize

In [0]:
@use_named_args(space)
def objective(**params):
    accuracy = 0
    for train_index, test_index in kf.split(X):
        # Make split
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Setup Data Generators based on split
        training_generator = DataGenerator(X_train, labels,
                                           batch_size=batch_size,
                                           dim=DATA_SHAPE,
                                           n_classes=NB_CLASSES,
                                           data_directory=DATA_PATH, 
                                           shuffle=True,
                                           data_type="Training")
        validation_generator = DataGenerator(X_test, labels,
                                             batch_size=batch_size,
                                             dim=DATA_SHAPE,
                                             n_classes=NB_CLASSES,
                                             data_directory=DATA_PATH,
                                             shuffle=False,
                                             data_type="Validation")
        
        # Make model
        wnc = WaveNetClassifier(DATA_SHAPE, (NB_CLASSES,), kernel_size = kernel_size,
                        dilation_depth = dilation_depth, n_filters = n_filters,
                        pool_size_1=pool_size_1, pool_size_2=pool_size_2)
        model = wnc.get_model()
        
        # Compile model
        model.compile("adam", loss='categorical_crossentropy', metrics=['accuracy'])

        
        model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=9,
                    epochs=epochs,
                    callbacks=callbacks_list,
                    initial_epoch=start_epoch)
        
        acc = model.evaluate(X_test, y_test, verbose=1)[1]
        accuracy += acc
    avg_acc = accuracy/NB_OF_FOLDS
    return -avg_acc # return negative as trying to minimize

### Perform Search

In [0]:
res_gp = gp_minimize(objective, space, n_calls=8, n_random_starts=2, random_state=0)

## Results

In [0]:
load_weights = False
weights_path = MODEL_PATH + "epoch-07-val_acc-0.14.hdf5"

start_epoch = 0
if load_weights:
    model = load_model(weights_path)
#     model.load_weights(weights_path)
    last_epoch = weights_path.split("-")[-3]
    start_epoch = int(last_epoch)

In [0]:
if not load_weights:
#     adam = optimizers.Adam(lr=0.01)


In [0]:
checkpoint = ModelCheckpoint(MODEL_PATH + "epoch-{epoch:02d}-val_acc-{val_acc:.2f}.hdf5",
                             monitor='val_acc', verbose=0, save_best_only=False,
                             save_weights_only=False, mode='auto', period=1)

csv_logger = CSVLogger(RESULTS_PATH + "test_8.csv", append=True)
callbacks_list = [checkpoint, csv_logger]